The data is currently stored as `.xls` files. In this notebook, we will implement some code to manipulate the data as `pandas.Dataframes` and store as more efficient `.parquet` files on disk.

In [3]:
# import any required libraries here
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
#connect google drive and google collab
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install --upgrade xlrd

First, we need to read the `.xls` files into `pandas.Dataframes`. You can use [pandas.read_excel](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for this.

In [ ]:
# load the building data 
# consider the different number of header rows!
# only load same header rows 

def load_data(filepath1, filepath2, col1=None,col2=None):
    '''
    Merging two .xls-files and saving as a pandas.Dataframe
    
    Load the two .xls-files, create a unique column and using this unique 
    column as a header. Furthermore rename the column OBIS Bezeichnung_O_reibung
    to 'Date' and change the type to pd.datetime. Next change type of remaining 
    columns to numeric. Finally merge both .xls-files and save as a pandas.Dataframe

    Parameters:
    filepath1 (string): Filepath of first .xls-Document
    filepath2 (string): Filepath of second .xls-Document 
    col1 (string): optional parameter, which states the columns to read in file1 
    col2 (string): optional parameter, which states the columns to read in file2           

    Returns:
    df_merged (pd.Dataframe): Merged Dataframe 
    
    '''
    #read data 
    file1 = pd.read_excel(filepath1,usecols=col1, header=1)
    file2 = pd.read_excel(filepath2,usecols=col2, header=1)
    #dict with OBIS Beschreibung and abbreviation
    list_dic = {'Betriebsstunden':'Betriebsstd', 'Fehlerstunden':'Fehlerstd', 'Vorlauftemperatur':'Vorlauftmp',
        'Wärmeenergie total':'Wtotal','Temperaturdifferenz': 'TmpDiff','Volumen Kanal 1':'VK1',
        'WV+ Arbeit Tarif 1':'WV+T1','WV- Arbeit Tarif 1':'WV-T1','WV+ Arbeit tariflos':'WV+tariflos',
        'WV- Arbeit tariflos':'WV-tariflos', 'WV+ Momentanwert Tariflos':'WV+ Momtrflos','Fehler Flags':'FehFlag',
        'BV- Arbeit Tarif 1': 'BV-T1','BV+ Arbeit Tarif 1':'BV+T1','Wärmeleistung':'Wleistung',
        'BV+ Arbeit tariflos':'BV+tariflos','BV- Arbeit tariflos':'BV-tariflos', 'Durchfluss':'Dfluss',
        'Wärmeenergie Tarif 1':'WTarif1', 'P Summe' : 'PSum', 'Rücklauftemperatur':'Rücklauftmp', 'Volumen':'Vol'}
    file1.iloc[2].replace(list_dic, inplace=True)
    file2.iloc[2].replace(list_dic, inplace=True)
    #create unique identifier
    for column in file1:
      file1[column].iloc[2] = file1[column].iloc[1][0] + '_'+file1[column].iloc[0]+ '_'+file1[column].iloc[2]
    for column in file2:
      file2[column].iloc[2] = file2[column].iloc[1][0] + '_'+file2[column].iloc[0]+ '_'+file2[column].iloc[2]
    #change header to unique identififer  
    file1.columns = file1.iloc[2]
    file2.columns = file2.iloc[2]
    #drop not useful rows
    file1 = file1.drop([0,1,2,3])
    file2 = file2.drop([0,1,2,3])
    #rename col OBIS Bezeichnung_O_reibung to date 
    file1 = file1.rename(columns = {'O_Beschreibung_OBIS Bezeichnung':'Date'})
    file2 = file2.rename(columns = {'O_Beschreibung_OBIS Bezeichnung':'Date'})
    #change type of col date to date format  
    file1['Date'] = pd.to_datetime(file1['Date'])
    file2['Date'] = pd.to_datetime(file2['Date']) 
    #change type of remaining columns to numeric 
    file1.iloc[:,1:] = file1.iloc[:,1:].astype('float')
    file2.iloc[:,1:] = file2.iloc[:,1:].astype('float')
    #sort data
    file1 = file1.sort_values(by='Date')
    file2 = file2.sort_values(by='Date')
    #take difference for specific cols per building
    #file1_diff = file1.iloc[:,1:].diff()
    #file1_diff.insert(0,'Date', file1['Date'])
    #file2_diff = file2.iloc[:,1:].diff()
    #file2_diff.insert(0,'Date', file2['Date'])
    #merge the two files (considering reseting index, removing overlapping data in date)
    df_merged = pd.concat([file1_diff, file2_diff]).reset_index(drop=True)
    df_merged = df_merged.drop_duplicates(subset='Date', keep='first')

    return df_merged

In [ ]:
path = '/content/drive/MyDrive/case_studies_data/'
#excluded empty columns and col Gerätenummer, which is not of interest
oh12 = load_data(path+'OH12.xls',path+ 'OH12_01_26-07_19.xls',"A:K,M:T,V:W,Y,AA")
oh14 = load_data(path+'OH14.xlsx',path+ 'OH14_01_26-07_19.xls',"A:O,Q,R")
chemie = load_data(path+'Chemie.xls',path+ 'Chemie_01_26-07_19.xls',col1="A:H,J,M,P,R",col2="A:H,J:N")
kita= load_data(path+'Kita Hokido.xls',path+ 'Kita Hokido_05_22_20-07_19_22.xls')
hg2= load_data(path+'HG II.xls',path+ 'HGII_01_26-07_19.xls',"A:Q,S,U:W")
tagespflege = load_data(path+'Großtagespflege.xls',path+'Grosstagespflege_04_05-07_19.xls',col1="A:T,V,X,Z,AB,AD,AE,AG,AI,AK,AM,AN,AP,AR,AT", col2="A:S,U,W,Y")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Next, we need to implement a function that takes a `pandas.Dataframe` and a path string as an input and writes the data to disk as a `parquet` file. You can use the [PyArrow library](https://arrow.apache.org/docs/python/parquet.html) for this: 

In [ ]:
def write_as_parquet(df, path):
    '''
    Reading a pandas.Dataframe into a pandas table and saving
    it as a parquet file
    
    Parameters:
    df (pandas.Dataframe): Dataframe to be saved as .parquet
    path (string): Path where the parquet file should be saved           
    
    '''
    pq.write_table(pa.Table.from_pandas(df), path)


In [ ]:
write_as_parquet(oh14,path +'oh14.parquet' )
write_as_parquet(oh12,path +'oh12.parquet' )
write_as_parquet(chemie,path +'chemie.parquet' )
write_as_parquet(kita,path +'kita.parquet' )
write_as_parquet(hg2,path +'hg2.parquet' )
write_as_parquet(tagespflege,path +'tagespflege.parquet' )

Now we need the opposite functionality: a function that reads data from a `.parquet` file on disk and returns it as a `pandas.Dataframe`. Implement this function such that it can take a list of names of column to load as an _optional_ parameter. 

In [ ]:
def load_to_pandas(path, columns=None):
    '''
    Reading a parquet file and saving as a pandas.Dataframe with optional 
    parameter columns, which states the list of columns to read 
    
    Parameters:
    path (string): Path of parquet file 
    columns (optional parameter): columns to load         

    Returns:
    df (pd.Dataframe): parquet-file as pandas.Dataframe   
    
    '''
    df = pq.read_table(path, columns).to_pandas()
    return df

Great! We can now store data more efficiently on disk and know how to load it again. Store all the data we have as one `.parquet` file per building.